In [12]:
import requests
from dotenv import load_dotenv
import os
import inspect
import json
from groq import Groq

_ = load_dotenv()

In [13]:
def get_features(
            age: int = None,
            risk_rate: str = None,
            gender: str = None,
            completed_orders: str = None,
            canceled_orders: str = None,
            average_price: float = None,
            most_frequent_order_type: str = None,
            most_frequent_execution_status: str = None,
            most_frequent_sector_name: str = None,
            avg_order_rate_difference: str = None,
            avg_order_quantity_rate_difference: str = None,
):
    """
    Constructs a dict with all the client details specified in the user query.
    Args:
        age (int, optional): The age of the client. Must be a non-negative integer. If the age is unknown use 40 as the default value.
        risk_rate (str, optional): The risk rate of the client. Can only be one of the following with case sensitivity; 'High', 'Low', 'Medium', 'Not Assigned'. If the risk rate is unknown use 'Not Assigned' as the default value.
        gender (str, optional): the gender of the client. Can only be one of the following with case sensitivity; 'Male', 'Female'. If the gender is unknown use 'Male' as the default value.
        completed_orders (str, optional): The completed orders of the client. can only be one of the following with case sensitivity; 'All', 'More Than Half', 'Less Than Half', 'None'. If the completed orders is unknown use 'More Than Half' as the default value.
        canceled_orders_ratio (str, optional): The canceled orders of the client. Can only be one of the following with case sensitivity;'All', 'Most', 'Moderate', 'Little', 'None'. If the canceled orders is unknown use 'Moderate' as the default value.
        average_price (float, optional): The average price of the clients orders. If the average price is not known use 9.5 as the default value.
        most_frequent_order_type (str, optional): The most frequent order type of the client. Can only be one of the following with case sensitivity;'Buy', 'Sell'. If the order type is unknown use 'Sell' as the default value.
        most_frequent_execution_status (str, optional): The most frequent execution status of the clients orders. Can only be one of the following with case sensitivity; 'Executed', 'Not Executed', 'Partially Executed'. If the execution status is unknown use 'Executed' as the default value
        most_frequent_sector_name (str, optional): The most frequent sector that the clients orders are in. Can only be one of the following with case sensitivity;'Industries', 'Financials', 'Real Estate', 'Materials', 'Energy','INVESTMENT', 'Consumer Discretionary', 'INDUSTRIAL','Information Technology', 'Health Care', 'Consumer Staples','REAL ESTATE', 'Telecommunication Services', 'Basic Materials','Others', 'FOOD', 'Tourism', 'Telecommunications', 'SERVICES'. If the sector is unknown use 'Financials' as the default value.
        avg_order_rate_difference (str, optional): The change in the client's order activity. Can only be one of the following with case sensitivity; 'Increased', 'Decreased', 'Constant'. If the change in order rate is unknown use 'Constant' as the default value.
        avg_order_quantity_rate_difference (str, optional): The change in the client's order quantity. Can only be one of the following with case sensitivity; 'Increased', 'Decreased', 'Constant'. If the change in order quantity rate is unknown use 'Constant' as the default value.
    Returns:
        dict: The client data.
    """

    # Validations:
    valid_sectors = [
        "Industries",
        "Financials",
        "Real Estate",
        "Materials",
        "Energy",
        "INVESTMENT",
        "Consumer Discretionary",
        "INDUSTRIAL",
        "Information Technology",
        "Health Care",
        "Consumer Staples",
        "REAL ESTATE",
        "Telecommunication Services",
        "Basic Materials",
        "Others",
        "FOOD",
        "Tourism",
        "Telecommunications",
        "SERVICES",
    ]

    if age is not None and age < 0:
        age = None

    if risk_rate is not None and risk_rate not in [
        "High",
        "Low",
        "Medium",
        "Not Assigned",
    ]:
        risk_rate = None
    
    if gender is not None and gender not in ["Male", "Female"]:
        gender = None
    
    if completed_orders and completed_orders not in [
        "All",
        "More Than Half",
        "Less Than Half",
        "None",
    ]:
        completed_orders = None
    
    if canceled_orders and canceled_orders not in [
        "All",
        "Most",
        "Moderate",
        "Little",
        "None",
    ]:
        canceled_orders = None
    
    if average_price and average_price < 0:
        average_price = None
    
    if most_frequent_order_type and most_frequent_order_type not in ["Buy", "Sell"]:
        most_frequent_order_type = None
    
    if most_frequent_execution_status and most_frequent_execution_status not in [
        "Executed",
        "Not Executed",
        "Partially Executed",
    ]:
        most_frequent_execution_status = None
    
    if most_frequent_sector_name and most_frequent_sector_name not in valid_sectors:
        most_frequent_sector_name = None
    
    if avg_order_rate_difference and avg_order_rate_difference not in [
        "Increased",
        "Decreased",
        "Constant",
    ]:
        avg_order_quantity_rate_difference = None

    if avg_order_quantity_rate_difference and avg_order_quantity_rate_difference not in [
        "Increased",
        "Decreased",
        "Constant",
    ]:
        avg_order_quantity_rate_difference = None

In [14]:
def generate_function_info_json(func):
    """
    Generates a JSON document describing the given function.
    Args:
        func (callable): The function to inspect.
    Returns:
        dict: A dictionary representing the function information in the desired format.
    """
    if not callable(func):
        raise ValueError("Input must be a callable function.")

    # Extract function details
    func_name = func.__name__
    func_docstring = func.__doc__ or ""
    func_parameters = inspect.signature(func).parameters

    func_docstring = func_docstring.strip().split("\n")

    # Create the JSON structure
    json_data = {
        "type": "function",
        "function": {
            "name": func_name,
            "description": func_docstring[0],
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    }

    # Add function parameters to the JSON
    i = 0
    for param_name, param_info in func_parameters.items():
        json_data["function"]["parameters"]["properties"][param_name] = {
            "type": str(param_info.annotation).split("'")[1],
            "description": func_docstring[2+i].split(": ")[1],
        }
        i+=1
        json_data["function"]["parameters"]["required"].append(param_name)

    return json_data

In [15]:
calculate_info_json = generate_function_info_json(get_features)
print(json.dumps(calculate_info_json, indent=4))

{
    "type": "function",
    "function": {
        "name": "get_features",
        "description": "Constructs a dict with all the client details specified in the user query.",
        "parameters": {
            "type": "object",
            "properties": {
                "age": {
                    "type": "int",
                    "description": "The age of the client. Must be a non-negative integer. If the age is unknown use 40 as the default value."
                },
                "risk_rate": {
                    "type": "str",
                    "description": "The risk rate of the client. Can only be one of the following with case sensitivity; 'High', 'Low', 'Medium', 'Not Assigned'. If the risk rate is unknown use 'Not Assigned' as the default value."
                },
                "gender": {
                    "type": "str",
                    "description": "the gender of the client. Can only be one of the following with case sensitivity; 'Male', 'Female'. If 